In [1]:
import radiant as rad
import numpy as np
import matplotlib.pyplot as plt


# Problem Parameters
a = 0.
b = 1.


def exact(*x):
    return np.prod(np.sin(np.asarray(x) * 2 * np.pi / (b - a)), axis=0)


def f(*x):
    return (len(x) * (2 * np.pi / (b - a)) ** 2) * exact(*x)


def g(*x):
    return np.zeros_like(x[0])

In [2]:
# Parameters
d = 2
k = 2
delta = 0.5
Ns = np.arange(3, 15, 1, dtype=int)

# Computed Parameters
ranges = ((a, b),) * d

idx_funcs = [
    lambda cs: np.all([np.logical_and(c != a, c != b) for c in cs], axis=0),
    lambda cs: np.any([np.logical_or(c == a, c == b) for c in cs], axis=0),
]
operators = [
    lambda func: lambda *x: - func.laplacian(*x),
    lambda func: lambda *x: func(*x),
]

# Integration
error_integrator = rad.integrate.QuadIntegrator(ranges)

# Data collection
results = np.zeros((np.size(Ns), 3))

for i, N in enumerate(Ns):
    xc = rad.gridn(ranges, N, flat=True)

    # Solve for approximate solution
    phi = rad.Wendland(d, k, delta, xc)
    solver = rad.solve.CollocationSolver(phi, operators, idx_funcs)
    approx = solver.solve(f, g)
    error = rad.error(exact, approx, error_integrator)
    cond = solver.cond()
    bw = np.max(solver.bandwidth())
    
    results[i,0] = error 
    results[i, 1] = cond
    results[i, 2] = bw
    
    print("N:", N)
    print("  L2 Relative Error:", error)
    print("  Condition Number:", cond)
    print("  Bandwidth:", bw)

N: 3
  L2 Relative Error: 0.9798324515368192
  Condition Number: 3.816901038168581
  Bandwidth: 4
N: 4
  L2 Relative Error: 0.8597648614673191
  Condition Number: 13.630367214308798
  Bandwidth: 10
N: 5
  L2 Relative Error: 2.065576822828394
  Condition Number: 101.47586221589043
  Bandwidth: 17
N: 6
  L2 Relative Error: 2.052720425340487
  Condition Number: 178.7946778700187
  Bandwidth: 26
N: 7
  L2 Relative Error: 2.02153121883885
  Condition Number: 205.09457907163153
  Bandwidth: 38
N: 8
  L2 Relative Error: 2.079494601882365
  Condition Number: 801.8184412969515
  Bandwidth: 51
N: 9
  L2 Relative Error: 2.1046055272138773
  Condition Number: 3327.7765382117464
  Bandwidth: 66
N: 10
  L2 Relative Error: 2.150954492099158
  Condition Number: 15504.167917999986
  Bandwidth: 84
N: 11
  L2 Relative Error: 2.171910206219492
  Condition Number: 54288.5756863769
  Bandwidth: 103
N: 12
  L2 Relative Error: 2.1876440970440636
  Condition Number: 153740.51337370314
  Bandwidth: 124
N: 13
  

KeyboardInterrupt: 

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(12,3))

axs[0].set_xlabel(r"$N$")
axs[0].set_ylabel(r"$\| u - s\|_{L^{2}}$")
axs[0].set_yscale('log')
axs[0].plot(Ns ** d, results[:, 0], 'o-')

axs[1].set_xlabel(r"$N$")
axs[1].set_ylabel(r"$\kappa_{2}\left(A_{L}\right)$")
axs[1].set_yscale('log')
axs[1].plot(Ns ** d, results[:, 1], 'o-')

axs[2].set_xlabel(r"$N$")
axs[2].set_ylabel(r"$A_{L}$ Bandwidth")
axs[2].plot(Ns ** d, results[:, 2], 'o-')

plt.subplots_adjust(wspace=0.5)
plt.savefig("figures/c3-kansa-poisson-ns.pdf", bbox_inches='tight')

In [ ]:
np.shape(results)